### 리뷰분석 test

In [1]:
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
import os

dotenv_path = '../.env' 
load_dotenv(dotenv_path=dotenv_path)

db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')
db_connection_str = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
db_engine = create_engine(db_connection_str)

sql_query = "SELECT * FROM public.hyundai_segment_purchases;"

df = pd.read_sql(sql_query, db_engine)
df.head()


,Id,CarType,Manufacturer,Model,Age,Gender,Satisfaction,Review
0,41,SUV,현대,코나,60,남성,0.5,최악입니다. 다른 차량에서는 찾아볼 수 없는 휀다쪽 심한 단차로 인해 매일 스트레스...
1,42,중형,현대,쏘나타 디 엣지,30,남성,3.5,십년 이전 쏘나타 타다가 요번에 나온 차량 샀는데 신세계입니다. 우선은 디자인적으로...
2,43,대형,현대,그랜저 하이브리드,40,남성,5.0,아반떼MD를 약 14년간 오랜시간 별탈없이 타고 나이가 지긋하게 든 시점에서 그랜저...
3,44,중형,현대,쏘나타 디 엣지 하이브리드,50,남성,4.0,승차감.디자인.실내공간.연비효율.성능연에서 2년전 구입했던DN-8보다 월등히 탁월하...
4,45,SUV,현대,싼타페 하이브리드,50,남성,5.0,13년간 잘 타던 2010년형 아반떼md차량을 타다가 이번에 2025싼타페하이브리드...


In [ ]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
import re
from collections import Counter
from konlpy.tag import Okt
from openai import OpenAI
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 환경변수 로드
load_dotenv()

class CarReviewAnalyzer:
    def __init__(self, db_config):
        """
        차량 리뷰 분석기 초기화
        
        Args:
            db_config: 데이터베이스 연결 설정
        """
        # DB 연결
        self.engine = create_engine(
            f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['name']}"
        )
        
        # NLP 도구 초기화
        self.okt = Okt()
        
        # OpenAI API 초기화
        self.client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        
        # 분석 카테고리 정의
        self.categories = {
            '연비': ['연비', '기름', '주유', '리터', '연료', '효율', '경제적', '기름값', '주유비'],
            '편의기능': ['편의', '기능', '옵션', '장치', '시스템', '스마트', '자동', '편리', '기술'],
            '실내공간': ['공간', '실내', '넓은', '좁은', '트렁크', '시트', '좌석', '수납', '공간감'],
            '승차감': ['승차감', '진동', '소음', '정숙', '안정', '부드러운', '편안', '서스펜션', '주행'],
            '초기품질': ['품질', '고장', '문제', '하자', '수리', 'AS', '서비스', '내구성', '마감']
        }
        
        # 감성 분석용 키워드
        self.positive_words = ['좋다', '훌륭', '만족', '최고', '추천', '편리', '안정', '우수', '뛰어나다', 
                               '괜찮다', '나쁘지않다', '충분', '적당', '합리적', '경제적']
        self.negative_words = ['나쁘다', '별로', '불만', '실망', '부족', '아쉽다', '문제', '고장', 
                               '불편', '후회', '최악', '심각', '개선', '부실']

    def get_most_purchased_car(self, age, gender):
        """
        특정 연령대와 성별의 가장 많이 구매한 차량 조회
        
        Args:
            age: 나이
            gender: 성별
            
        Returns:
            model: 가장 많이 구매한 차량 모델
            reviews: 해당 차량의 리뷰 데이터
        """
        # 가장 많이 구매한 차량 조회
        query = f"""
        SELECT Model, COUNT(*) as count
        FROM public.hyundai_segment_purchases
        WHERE Age = {age} AND Gender = '{gender}'
        GROUP BY Model
        ORDER BY count DESC
        LIMIT 1;
        """
        
        result = pd.read_sql(query, self.engine)
        
        if result.empty:
            return None, None
            
        most_purchased_model = result.iloc[0]['model']
        
        # 해당 모델의 모든 리뷰 조회
        review_query = f"""
        SELECT *
        FROM public.hyundai_segment_purchases
        WHERE Model = '{most_purchased_model}'
        """
        
        reviews = pd.read_sql(review_query, self.engine)
        
        return most_purchased_model, reviews

    def analyze_sentiment(self, text):
        """
        텍스트의 감성 분석 (긍정/부정)
        
        Args:
            text: 분석할 텍스트
            
        Returns:
            sentiment: 'positive', 'negative', 'neutral'
            score: 감성 점수 (-1 ~ 1)
        """
        if pd.isna(text):
            return 'neutral', 0
            
        # 형태소 분석
        morphs = self.okt.morphs(text)
        
        # 긍정/부정 단어 카운트
        pos_count = sum(1 for word in morphs if word in self.positive_words)
        neg_count = sum(1 for word in morphs if word in self.negative_words)
        
        # 점수 계산
        if pos_count + neg_count == 0:
            return 'neutral', 0
            
        score = (pos_count - neg_count) / (pos_count + neg_count)
        
        if score > 0.2:
            return 'positive', score
        elif score < -0.2:
            return 'negative', score
        else:
            return 'neutral', score

    def extract_category_keywords(self, reviews, category):
        """
        특정 카테고리에 대한 키워드 추출
        
        Args:
            reviews: 리뷰 데이터프레임
            category: 분석 카테고리
            
        Returns:
            keywords: 추출된 키워드와 빈도
        """
        category_keywords = self.categories[category]
        relevant_reviews = []
        
        # 카테고리 관련 리뷰 필터링
        for review in reviews['Review'].dropna():
            if any(keyword in review for keyword in category_keywords):
                relevant_reviews.append(review)
        
        if not relevant_reviews:
            return []
        
        # 형태소 분석 및 명사 추출
        all_nouns = []
        for review in relevant_reviews:
            nouns = self.okt.nouns(review)
            # 2글자 이상 명사만 선택
            all_nouns.extend([noun for noun in nouns if len(noun) >= 2])
        
        # 빈도 계산
        noun_counts = Counter(all_nouns)
        
        # 상위 10개 키워드 반환
        return noun_counts.most_common(10)

    def generate_representative_review(self, model, reviews, category_analysis):
        """
        LLM을 활용한 대표 리뷰 생성
        
        Args:
            model: 차량 모델명
            reviews: 리뷰 데이터
            category_analysis: 카테고리별 분석 결과
            
        Returns:
            representative_review: 생성된 대표 리뷰
        """
        # 프롬프트 구성
        prompt = f"""
        다음은 {model} 차량에 대한 리뷰 분석 결과입니다.
        
        카테고리별 주요 키워드:
        """
        
        for category, keywords in category_analysis.items():
            if keywords:
                keyword_str = ", ".join([f"{word}({count}회)" for word, count in keywords[:5]])
                prompt += f"\n{category}: {keyword_str}"
        
        prompt += f"""
        
        위 분석 결과를 바탕으로 {model} 차량에 대한 종합적인 리뷰를 200자 이내로 작성해주세요.
        각 카테고리의 특징을 균형있게 포함하되, 실제 구매자의 관점에서 자연스럽게 작성해주세요.
        """
        
        try:
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "당신은 자동차 리뷰 전문가입니다. 객관적이고 균형잡힌 리뷰를 작성합니다."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=500,
                temperature=0.7
            )
            
            return response.choices[0].message.content
            
        except Exception as e:
            print(f"LLM 리뷰 생성 오류: {e}")
            return self.generate_simple_review(model, category_analysis)

    def generate_simple_review(self, model, category_analysis):
        """
        키워드 기반 간단한 리뷰 생성 (LLM 실패 시 대체)
        
        Args:
            model: 차량 모델명
            category_analysis: 카테고리별 분석 결과
            
        Returns:
            review: 생성된 리뷰
        """
        review_parts = [f"{model}은(는)"]
        
        for category, keywords in category_analysis.items():
            if keywords and len(keywords) > 0:
                top_keyword = keywords[0][0]
                review_parts.append(f"{category} 측면에서 {top_keyword}이(가) 자주 언급되며,")
        
        review = " ".join(review_parts)
        if review.endswith(","):
            review = review[:-1] + " 등이 특징적입니다."
            
        return review

    def analyze_reviews(self, age, gender):
        """
        전체 리뷰 분석 프로세스 실행
        
        Args:
            age: 나이
            gender: 성별
            
        Returns:
            analysis_result: 분석 결과 딕셔너리
        """
        # 1. 가장 많이 구매한 차량 조회
        model, reviews = self.get_most_purchased_car(age, gender)
        
        if model is None:
            return {"error": "해당 조건에 맞는 구매 데이터가 없습니다."}
        
        print(f"분석 대상 차량: {model}")
        print(f"총 리뷰 수: {len(reviews)}")
        
        # 2. 감성 분석
        sentiments = []
        for review in reviews['Review']:
            sentiment, score = self.analyze_sentiment(review)
            sentiments.append({'sentiment': sentiment, 'score': score})
        
        sentiment_df = pd.DataFrame(sentiments)
        sentiment_summary = {
            'positive': len(sentiment_df[sentiment_df['sentiment'] == 'positive']),
            'negative': len(sentiment_df[sentiment_df['sentiment'] == 'negative']),
            'neutral': len(sentiment_df[sentiment_df['sentiment'] == 'neutral']),
            'avg_score': sentiment_df['score'].mean()
        }
        
        # 3. 카테고리별 키워드 분석
        category_analysis = {}
        for category in self.categories.keys():
            keywords = self.extract_category_keywords(reviews, category)
            category_analysis[category] = keywords
        
        # 4. 대표 리뷰 생성
        representative_review = self.generate_representative_review(model, reviews, category_analysis)
        
        # 5. 결과 종합
        result = {
            'model': model,
            'total_reviews': len(reviews),
            'sentiment_analysis': sentiment_summary,
            'category_keywords': category_analysis,
            'representative_review': representative_review,
            'recommendation_score': self.calculate_recommendation_score(sentiment_summary, len(reviews))
        }
        
        return result

    def calculate_recommendation_score(self, sentiment_summary, total_reviews):
        """
        추천 점수 계산
        
        Args:
            sentiment_summary: 감성 분석 결과
            total_reviews: 전체 리뷰 수
            
        Returns:
            score: 추천 점수 (0-100)
        """
        if total_reviews == 0:
            return 50
            
        # 긍정 비율 계산
        positive_ratio = sentiment_summary['positive'] / total_reviews
        
        # 평균 감성 점수 반영
        avg_sentiment = (sentiment_summary['avg_score'] + 1) / 2  # 0-1로 정규화
        
        # 최종 점수 계산
        score = (positive_ratio * 0.7 + avg_sentiment * 0.3) * 100
        
        return round(score, 1)

# 사용 예제
if __name__ == "__main__":
    # DB 설정
    db_config = {
        'host': os.getenv('DB_HOST', 'localhost'),
        'port': os.getenv('DB_PORT', '5432'),
        'user': os.getenv('DB_USER', 'postgres'),
        'password': os.getenv('DB_PASSWORD', ''),
        'name': os.getenv('DB_NAME', 'carfin')
    }
    
    # 분석기 초기화
    analyzer = CarReviewAnalyzer(db_config)
    
    # 20대 남성 대상 분석 실행
    result = analyzer.analyze_reviews(age=20, gender='남성')
    
    # 결과 출력
    print("\n" + "="*50)
    print(" 차량 리뷰 분석 결과")
    print("="*50)
    
    print(f"\n 추천 차량: {result['model']}")
    print(f" 총 리뷰 수: {result['total_reviews']}")
    
    print("\n 감성 분석:")
    print(f"  - 긍정: {result['sentiment_analysis']['positive']}개")
    print(f"  - 부정: {result['sentiment_analysis']['negative']}개")
    print(f"  - 중립: {result['sentiment_analysis']['neutral']}개")
    print(f"  - 평균 점수: {result['sentiment_analysis']['avg_score']:.2f}")
    
    print("\n 카테고리별 주요 키워드:")
    for category, keywords in result['category_keywords'].items():
        if keywords:
            keyword_str = ", ".join([f"{word}" for word, count in keywords[:3]])
            print(f"  - {category}: {keyword_str}")
    
    print(f"\n 추천 점수: {result['recommendation_score']}점/100점")
    
    print("\n 대표 리뷰:")
    print(result['representative_review'])